# Example: MOOSE wrapper in fiberis

Shenyao Jin, shenyaojin@mines.edu

`fiberis` provides a module to simulate the pressure diffusion using HMM(Hydro-Mechanical Model). `fiberis.moose` is able to control the whole simulation without interacting with the terminal and annoying `.i`, `.e` and `.csv` files!

For this script, I'm going to give a full simulation example so that you can understand the full workflow of `fiberis.moose` module.

In [ ]:
import os

from DSS_analyzer_Mariner.Data1D_GAUGE import Data1D_GAUGE
%load_ext autoreload
%autoreload 2

Dependence: **MOOSE**.

If you don't know how to install it, refer to my [note](https://shenyaojin.github.io/theory/fiberis-setup/), it will provide instructions on how to install the MOOSE env on UNIX-like systems.

## Import libs

`fiberis.moose` has multiple libs playing different roles of the whole simulation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

In [18]:
# fiberis modules
from fiberis.moose.model_builder import ModelBuilder
from fiberis.moose.runner import MooseRunner
from fiberis.moose.config import (
    MatrixConfig, SRVConfig, HydraulicFractureConfig, ZoneMaterialProperties,
    SimpleFluidPropertiesConfig, PointValueSamplerConfig, LineValueSamplerConfig, TimeSequenceStepper
)
from fiberis.analyzer.Data1D.Data1D_Gauge import Data1DGauge

## Problem statement

In this case, the PDE problem is a typical and easy problem I will encounter in his research: coupled pressure-strain diffusion:

1. mesh: `200 m` * `200 m` (length * width).
2. Physical field:
    - Hydraulic fracture (HF) has the highest permeability `1e-12` $m^2$, height `0.2 m` width `50 m`. At the center of mesh.
    - Stimulated Reservoir Volume (SRV) has lower permeability, `1e-15` $m^2$, height `20 m` width `80m`. Around the fracture
    - Matrix has the lowest permeability, `1e-17` $m^2$. All other areas not covered by HF and SRV
    - The media flowing in the grid is **water**.
3. 1x injection (source, or boundary condition (B.C.)): at the center of HF, it's defined in pressure (psi or Pa, not in volume).
4. Params to solve: (1). The pressure field, (2) The strain tensor field which is represented in $\varepsilon_{xx}$ and $\varepsilon_{yy}$.
5. Governing equation: the classic HM model. Delete the thermal term in [this paper](https://www.sciencedirect.com/science/article/pii/S1365160909001695). I'm too lazy to use LaTeX to type all the equations. :P
6. Boundaries: 4x Neumann B.C (left, right, top, bottom) and forced pressure at the center of HF.
7. Postprocessor: I'll observe the pressure/strain perpendicular to fracture, and also in multiple points at my interested areas.
8. Executioner: ConstantDT, with mumps preconditioner. The solver is default: `Newton`.
9. Output: the simulation will output a series of `.csv` files, which can be handled by `fiberis` easily.

## Step 1: I/O

First, we need to tell `fiberis` where the results will be stored, where the source `.npz` files (in this case, all data are synthetic) will be read in.

In [6]:
output_dir = "../../output/moose_simulator"
os.makedirs(output_dir, exist_ok=True) # The location of the postprocessor files
input_file_name = os.path.join(output_dir, "example_moose_input_file.i") # Although you don't need to interact with the input file, still needed for the frame.

In [7]:
builder = ModelBuilder(project_name="example_moose") # Select the name you like.

## Step 2: Mesh builder

`fiberis` can generate the moose automatically if you gave the correct parameters. You don't need to set every block manually like what you did in `.i` file.

In [8]:
builder.build_stitched_mesh_for_fractures(
    fracture_y_coords = 0, # the fracture location(s). Make it a list if you want to simulate multiple HFs.
    domain_bounds = (-100, 100), # the total width of the mesh, 200 m as I mentioned in the problem statement, I like to put it like frac = 0, domain bounds = -100 -> 100; you can also set it like frac = 100, bound = 0 -> 200 which would also be fine.
    domain_length = 200, # the toal length of the mesh
    nx = 200, # How many grid you would like to set for the mesh, in **x/length** direction
    ny_per_layer_half = 40, # How many (grid / 2) you would like to set for the mesh, in **y/width** direction
    bias_y = 1.2, # the bias coefficient for y-axis sampling. bias = 1.2 means if the dy_{n-1} = 1 m, then dy_{n} = 1.2 m
)

Info: Successfully built stitched base mesh 'final_stitch_0'.


## Step 3: add elements to the mesh
`fiberis` focus on fracture and SRVs and is able to add them to grid in simple commands.

Here are what we need to do: (1) define the properties of each material, (2) fill out the mesh  with material in (1).

In [9]:
# Define porous materials.
matrix_mats = ZoneMaterialProperties(porosity=0.01, permeability="'1E-17 0 0  0 1E-17 0  0 0 1E-17'")
srv_mats = ZoneMaterialProperties(porosity=0.1, permeability="'1E-15 0 0  0 1E-15 0  0 0 1E-15'")
fracture_mats = ZoneMaterialProperties(porosity=0.1, permeability="'1E-12 0 0  0 1E-12 0  0 0 1E-12'")

In [11]:
center_x_val = 100 # it's domain_length/2, just because I want to put the elements at the center of grid

geometries = [
    SRVConfig(name="SRV", length=80, height=20, center_x=center_x_val, center_y=0, materials=srv_mats),
    HydraulicFractureConfig(name="Fracture", length=50, height=0.2, center_x=center_x_val, center_y=0, materials=fracture_mats)
]

# Fill out the mesh. NOTE: sort them first. The fracture must be set after SRV, or the larger SRV will cover the HF while doing simulation.
sorted_geometries = sorted(geometries, key=lambda x: x.height, reverse=True)
next_block_id = 1
for geom_config in sorted_geometries:
    if isinstance(geom_config, SRVConfig): # If the geometry is SRV, the do SRV configuration
        builder.add_srv_config(geom_config)
        builder.add_srv_zone_2d(geom_config, target_block_id=next_block_id)
    elif isinstance(geom_config, HydraulicFractureConfig): # If the geometry is HF, the do hydraulic fracture configuration
        builder.add_fracture_config(geom_config)
        builder.add_hydraulic_fracture_2d(geom_config, target_block_id=next_block_id)
    next_block_id += 1

In [13]:
# Add the injection/pressure B.C. to the HF
builder.add_nodeset_by_coord(
    nodeset_op_name = "injection", # The name display in DOT "i" file
    new_boundary_name="injection", # The boundary name
    coordinates=(center_x_val, 0, 0)
)

In [17]:
# Tel
biot_coeff_val = 0.7
fluid_props = SimpleFluidPropertiesConfig(name="water", bulk_modulus=2.2E9, viscosity=1.0E-3, density0=1000.0)
# Viscosity: 1e-3 pa*s
builder.add_fluid_properties_config(fluid_props)
builder.add_poromechanics_materials(
    fluid_properties_name="water",
    biot_coefficient=biot_coeff_val,
    solid_bulk_compliance=2E-11
)

Info: Added SimpleFluidProperties 'water'.
Info: Added poromechanics materials based on stored configurations.


## Add physics to the PDE problem

After defining the mesh in the problem, then is to tell the simulator what physical process is going during the simulation.

In [14]:
# What
builder.add_variables([
    {"name": "pp", "params": {"initial_condition": 0}}, # Initial pressure ~ let's do 0 at this case.
    {"name": "disp_x", "params": {"initial_condition": 0}},
    {"name": "disp_y", "params": {"initial_condition": 0}}
])

Info: Added 3 variables.


In [15]:
# Dictators
builder.set_porous_flow_dictator(dictator_name="dictator", porous_flow_variables="pp")
builder.add_global_params({"PorousFlowDictator": "dictator", "displacements": "'disp_x disp_y'"})

Info: Added/Updated UserObject 'dictator'.
Info: Added [GlobalParams] block.


In [16]:
# Add kernels to the simulator. Here are some typical kernels used in HMM.
builder.add_porous_flow_mass_time_derivative_kernel(kernel_name="dt", variable="pp")
builder.add_porous_flow_darcy_base_kernel(kernel_name="flux", variable="pp")
builder.add_stress_divergence_tensor_kernel(kernel_name="grad_stress_x", variable="disp_x", component=0)
builder.add_stress_divergence_tensor_kernel(kernel_name="grad_stress_y", variable="disp_y", component=1)
builder.add_porous_flow_effective_stress_coupling_kernel(kernel_name="eff_stress_x", variable="disp_x", component=0, biot_coefficient=biot_coeff_val)
builder.add_porous_flow_effective_stress_coupling_kernel(kernel_name="eff_stress_y", variable="disp_y", component=1, biot_coefficient=biot_coeff_val)
builder.add_porous_flow_mass_volumetric_expansion_kernel(kernel_name="mass_exp", variable="pp")

In [23]:
# For the pressure in HFs, use synthetic data
pressure_time = np.linspace(0, 3600, 72) # 72 points, each one is 50s
pressure_val = np.linspace(0, 5000, 72)

dataframe_injection = Data1DGauge()
dataframe_injection.taxis = pressure_time
dataframe_injection.data = pressure_val
dataframe_injection.start_time = datetime.datetime(2025, 1, 1, 0, 0, 0)
dataframe_injection.name = "injection"

In [24]:
# Add this array to the simulator, and it will be the B.C. I will use in the next step
builder.add_piecewise_function_from_data1d(name="injection_pressure_func", source_data1d=dataframe_injection)

Info: Added PiecewiseConstant Function 'injection_pressure_func' from Data1D source.


In [25]:
# Set up all the BC, including the HF pressure.
builder.set_hydraulic_fracturing_bcs(
    injection_well_boundary_name="injection", # those injection location(s). Use space to separate different HF injection names.
    injection_pressure_function_name="injection_pressure_func", # The name of injection, mentioned in the last block
    confine_disp_x_boundaries="left right",
    confine_disp_y_boundaries="top bottom"
    # left, right, top, bottom are predefined in the simulator.
)

Info: Added Boundary Condition 'injection_pressure'.
Info: Added Boundary Condition 'confinex'.
Info: Added Boundary Condition 'confiney'.
Info: Set standard hydraulic fracturing BCs using predefined set.


In [26]:
# Add aux kernels. In most cases we don't need to do any modifications on it. Just tell the simulator how to convert the displacement to strain & stress.
builder.add_standard_tensor_aux_vars_and_kernels({"stress": "stress", "total_strain": "strain"})

Info: Added standard AuxVariables and AuxKernels for tensors: ['stress', 'total_strain']


## Add monitor/postprocessor to the whole problem
After configuring the physics happened in that mesh, we need to sample some of the places which we are interested in. Here I'll add a linear sampler perpendicular to the HF, then do another point sampler near the HF.

In [ ]:
builder.add_postprocessor(
    PointValueSamplerConfig(name = "upper_point_sampler", # The name of the sampler, better not use space!
                            variable = "pp",  # The variable I want to monitor; see variables. The aux variables can be also monitored.
                            point = (center_x_val, 30, 0) # Location of the sampler.
                            )
)

builder.add_postprocessor(
    PointValueSamplerConfig(name = "lower_point_sampler",
                            variable = "strain_yy",
                            point = (center_x_val, -30, 0)
                            )
)

builder.add_postprocessor(
    LineValueSamplerConfig( # The linear sampler
        name = "strain_vertical",
        variable = "strain_yy",
        start_point = (center_x_val, -50, 0),
        end_point = (center_x_val, 50, 0),
        num_points = 200,
        other_params = {'sort_y': 'y'}
    )
)

In [ ]:
# Get the time info of the full simulation
total_time = dataframe_injection.taxis[-1] - dataframe_injection.taxis[0]
dt_control_func = TimeSequenceStepper()
dt_control_func.from_data1d(dataframe_injection)

In [ ]:
# Time stepper, I will comment the constDT method, use TimeSequenceStepper
# builder.add_executioner_block(
#     end_time = total_time,
#     dt = total_time / 50,
#     time_stepper_type = 'ConstantDT'
# )

builder.add_executioner_block(
    end_time = total_time,
    dt = total_time / 50, # I remember in sequence stepper you don't need a dt
    time_stepper_type = 'TimeSequenceStepper',
    stepper_config = dt_control_func
)